In [1]:
!pip install autoawq
!pip install nvidia-ml-py3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.3/84.3 kB 1.3 MB/s eta 0:00:00eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.4/33.4 MB 45.0 MB/s eta 0:00:00:00:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for nvidia-ml-py3: filename=nvidia_ml_py3-7.352.0-py3-none-any.whl size=19171 sha256=48ee14fdb72fb87eb055383b00842d21c6f3fca0dc629ca5234451c7bd435393
  Stored in directory: /root/.cache/pip/wheels/5c/d8/c0/46899f8be7a75a2ffd197a23c8797700ea858b9b34819fbf9e
Successfully built nvidia-ml-py3


In [2]:
%%capture
from awq import AutoAWQForCausalLM
from transformers import AutoTokenizer
import torch
from huggingface_hub import notebook_login as hfl

2024-05-26 16:11:50.782313: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-26 16:11:50.782438: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-26 16:11:50.933626: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


KeyboardInterrupt: 

In [ ]:
hfl()

In [ ]:
model_id = r"0xVolt/Meta-Llama-3-8B-Instruct-Hermes-2-Pro-SLERP"
quantModelPath = r"Meta-Llama-3-8B-Instruct-Hermes-2-Pro-SLERP-AWQ-4-bit"
quantConfig = {
    "q_group_size": 128,
    "w_bit": 4
}

Accidentally used `AutoModelForCausalLM` as a consequence of muscle memory. Should be `AutoAWQForCausalLM` :)

In [ ]:
model = AutoAWQForCausalLM.from_pretrained(
        model_id,
        torch_dtype=torch.float16,
        device_map="auto",
        trust_remote_code=True
)

tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True)

In [ ]:
%%time
model.quantize(tokenizer, quant_config=quantConfig)

In [ ]:
%%time
model.save_quantized("./" + quantModelPath, safetensors=True)
tokenizer.save_pretrained("./" + quantModelPath)